In [5]:
import pandas as pd
import numpy as np

# 读取complete stock data
df_complete = pd.read_csv('/Users/wenjiaqi/Downloads/bloomberg-competition/src/trading_system/data/complete_stock_data.csv')

print("原始数据形状:", df_complete.shape)
print("\n原始数据列名:")
print(df_complete.columns.tolist())
print("\n前5行数据:")
df_complete.head()


原始数据形状: (10369, 12)

原始数据列名:
['source_sheet', 'row_index', 'Ticker', 'Short Name', 'P_B', 'Percentile Rank_Current P_B_', 'Market Cap _USD_', 'Percentile Rank_Current Market Cap_', 'Market Cap', 'Price_D_1', 'ticker_clean', 'ticker_original']

前5行数据:


,source_sheet,row_index,Ticker,Short Name,P_B,Percentile Rank_Current P_B_,Market Cap _USD_,Percentile Rank_Current Market Cap_,Market Cap,Price_D_1,ticker_clean,ticker_original
0,DM_LG,1,NVDA US Equity,NVIDIA CORP,45.790287,100.0,4.576176e+12,100.0,4.576176e+12,188.32,NVDA,NVDA US Equity
1,DM_LG,2,MSFT US Equity,MICROSOFT CORP,11.125710,94.0,3.821019e+12,100.0,3.821019e+12,514.05,MSFT,MSFT US Equity
2,DM_LG,3,AAPL US Equity,APPLE INC,55.892689,100.0,3.675371e+12,100.0,3.675371e+12,247.66,AAPL,AAPL US Equity
3,DM_LG,4,GOOGL US Equity,ALPHABET INC-A,8.142908,90.0,2.955618e+12,100.0,2.955618e+12,244.15,GOOGL,GOOGL US Equity
4,DM_LG,5,GOOG US Equity,ALPHABET INC-C,8.142908,90.0,2.955618e+12,100.0,2.955618e+12,244.64,GOOG,GOOG US Equity


In [6]:
# 查看目标格式的示例
df_target = pd.read_csv('/Users/wenjiaqi/Downloads/bloomberg-competition/src/trading_system/data/holdings_cleaned.csv')

print("目标格式数据形状:", df_target.shape)
print("\n目标格式列名:")
print(df_target.columns.tolist())
print("\n目标格式前5行数据:")
df_target.head()


目标格式数据形状: (10414, 8)

目标格式列名:
['ticker', 'source_etf', 'country_code', 'region', 'section', 'weight', 'shares', 'price']

目标格式前5行数据:


,ticker,source_etf,country_code,region,section,weight,shares,price
0,NVDA,WLS,UW,NaN,Information Technology,4.330869,23444.971,192.57
1,MSFT,WLS,UW,NaN,Information Technology,3.723246,7429.881,522.40
2,AAPL,WLS,UW,NaN,Information Technology,3.562665,14619.625,254.04
3,AVGO,WLS,UW,NaN,Information Technology,1.526520,4612.342,345.02
4,2330,WLS,TT,NaN,Information Technology,0.837250,18536.260,1440.00


In [7]:
# 数据转换函数
def convert_complete_stock_data(df):
    """
    将complete stock data转换为目标格式
    """
    # 创建新的DataFrame
    df_converted = pd.DataFrame()
    
    # 提取ticker（从Ticker列中提取，去掉" US Equity"等后缀）
    df_converted['ticker'] = df['Ticker'].str.replace(' US Equity', '', regex=False)
    df_converted['ticker'] = df_converted['ticker'].str.replace(' Equity', '', regex=False)
    
    # 添加source_etf列（可以根据需要设置）
    df_converted['source_etf'] = 'COMPLETE_DATA'  # 或者根据source_sheet设置
    
    # 添加country_code列
    # 从ticker中提取国家代码，如果是US股票则设置为'US'
    df_converted['country_code'] = 'US'  # 默认都是US股票
    
    # 添加region列（暂时为空）
    df_converted['region'] = ''
    
    # 添加section列（可以根据需要设置）
    df_converted['section'] = 'Unknown'  # 或者根据其他信息推断
    
    # 添加其他列（从原始数据中提取）
    df_converted['weight'] = np.nan  # 权重信息在原始数据中没有
    df_converted['shares'] = np.nan  # 股数信息在原始数据中没有
    df_converted['price'] = df['Price_D_1']  # 使用Price_D_1作为价格
    
    # 添加原始数据中的其他有用列
    df_converted['market_cap'] = df['Market Cap _USD_']
    df_converted['p_b'] = df['P_B']
    df_converted['short_name'] = df['Short Name']
    df_converted['source_sheet'] = df['source_sheet']
    df_converted['row_index'] = df['row_index']
    
    return df_converted

# 执行转换
df_converted = convert_complete_stock_data(df_complete)

print("转换后数据形状:", df_converted.shape)
print("\n转换后数据列名:")
print(df_converted.columns.tolist())
print("\n转换后前5行数据:")
df_converted.head()


转换后数据形状: (10369, 13)

转换后数据列名:
['ticker', 'source_etf', 'country_code', 'region', 'section', 'weight', 'shares', 'price', 'market_cap', 'p_b', 'short_name', 'source_sheet', 'row_index']

转换后前5行数据:


,ticker,source_etf,country_code,region,section,weight,shares,price,market_cap,p_b,short_name,source_sheet,row_index
0,NVDA,COMPLETE_DATA,US,,Unknown,NaN,NaN,188.32,4.576176e+12,45.790287,NVIDIA CORP,DM_LG,1
1,MSFT,COMPLETE_DATA,US,,Unknown,NaN,NaN,514.05,3.821019e+12,11.125710,MICROSOFT CORP,DM_LG,2
2,AAPL,COMPLETE_DATA,US,,Unknown,NaN,NaN,247.66,3.675371e+12,55.892689,APPLE INC,DM_LG,3
3,GOOGL,COMPLETE_DATA,US,,Unknown,NaN,NaN,244.15,2.955618e+12,8.142908,ALPHABET INC-A,DM_LG,4
4,GOOG,COMPLETE_DATA,US,,Unknown,NaN,NaN,244.64,2.955618e+12,8.142908,ALPHABET INC-C,DM_LG,5


In [8]:
# 检查转换结果
print("转换后的数据统计:")
print(f"总行数: {len(df_converted)}")
print(f"唯一ticker数: {df_converted['ticker'].nunique()}")
print(f"country_code分布:")
print(df_converted['country_code'].value_counts())

print("\n转换后的数据示例:")
df_converted[['ticker', 'country_code', 'price', 'market_cap']].head(10)


转换后的数据统计:
总行数: 10369
唯一ticker数: 10146
country_code分布:
country_code
US    10369
Name: count, dtype: int64

转换后的数据示例:


,ticker,country_code,price,market_cap
0,NVDA,US,188.32,4.576176e+12
1,MSFT,US,514.05,3.821019e+12
2,AAPL,US,247.66,3.675371e+12
3,GOOGL,US,244.15,2.955618e+12
4,GOOG,US,244.64,2.955618e+12
5,AMZN,US,220.07,2.347027e+12
6,META,US,715.70,1.797940e+12
7,AVGO,US,356.70,1.684468e+12
8,TSLA,US,435.90,1.449433e+12
9,2330 TT,US,1415.00,1.203781e+12


In [9]:
# 保存转换后的数据
output_path = '/Users/wenjiaqi/Downloads/bloomberg-competition/src/trading_system/data/complete_stock_data_converted.csv'
df_converted.to_csv(output_path, index=False)

print(f"转换后的数据已保存到: {output_path}")
print(f"文件大小: {len(df_converted)} 行")

# 显示最终的数据结构
print("\n最终数据结构:")
print(df_converted.dtypes)
print("\n最终数据示例:")
df_converted.head()


转换后的数据已保存到: /Users/wenjiaqi/Downloads/bloomberg-competition/src/trading_system/data/complete_stock_data_converted.csv
文件大小: 10369 行

最终数据结构:
ticker           object
source_etf       object
country_code     object
region           object
section          object
weight          float64
shares          float64
price           float64
market_cap      float64
p_b             float64
short_name       object
source_sheet     object
row_index         int64
dtype: object

最终数据示例:


,ticker,source_etf,country_code,region,section,weight,shares,price,market_cap,p_b,short_name,source_sheet,row_index
0,NVDA,COMPLETE_DATA,US,,Unknown,NaN,NaN,188.32,4.576176e+12,45.790287,NVIDIA CORP,DM_LG,1
1,MSFT,COMPLETE_DATA,US,,Unknown,NaN,NaN,514.05,3.821019e+12,11.125710,MICROSOFT CORP,DM_LG,2
2,AAPL,COMPLETE_DATA,US,,Unknown,NaN,NaN,247.66,3.675371e+12,55.892689,APPLE INC,DM_LG,3
3,GOOGL,COMPLETE_DATA,US,,Unknown,NaN,NaN,244.15,2.955618e+12,8.142908,ALPHABET INC-A,DM_LG,4
4,GOOG,COMPLETE_DATA,US,,Unknown,NaN,NaN,244.64,2.955618e+12,8.142908,ALPHABET INC-C,DM_LG,5


In [10]:
import pandas as pd
import re
from typing import Optional, Tuple

# ============= 配置 =============
SOURCE_ETF = 'WLS'  # 固定值
INPUT_FILE = 'holdings.xlsx'  # 你的文件名
OUTPUT_FILE = 'holdings_cleaned.csv'

# ============= 辅助函数 =============

def is_section_row(row: pd.Series) -> bool:
    """判断是否为section行：Name列包含'members'且Ticker为空"""
    ticker_empty = pd.isna(row.get('Ticker')) or str(row.get('Ticker', '')).strip() == ''
    name = str(row.get('Name', ''))
    return ticker_empty and 'members' in name.lower()

def extract_section_name(text: str) -> str:
    """提取section名称，去掉括号内容
    Example: 'Information Technology (1362 members)' -> 'Information Technology'
    """
    match = re.search(r'^(.+?)\s*\(', text)
    if match:
        return match.group(1).strip()
    return text.strip()

def extract_ticker_and_country(ticker_text: str) -> Tuple[Optional[str], Optional[str]]:
    """从 'NVDA UW Equity' 提取 ('NVDA', 'UW')"""
    if pd.isna(ticker_text):
        return None, None
    
    parts = str(ticker_text).strip().split()
    if len(parts) >= 2:
        return parts[0], parts[1]  # ticker, country_code
    elif len(parts) == 1:
        return parts[0], None
    return None, None

def clean_numeric(value) -> Optional[float]:
    """清理数字：移除逗号，转换为float"""
    if pd.isna(value):
        return None
    try:
        # 移除逗号并转换
        cleaned = str(value).replace(',', '')
        return float(cleaned)
    except:
        return None

# ============= 主处理逻辑 =============

def process_excel_to_csv(input_file: str, output_file: str, source_etf: str):
    """主函数：处理Excel并输出CSV"""
    
    # 1. 读取数据
    print(f"读取文件: {input_file}")
    df = pd.read_excel(input_file)
    print(f"原始数据: {len(df)} 行")
    
    # 2. 准备结果容器
    results = []
    current_section = None
    
    # 3. 逐行处理（状态机）
    for idx, row in df.iterrows():
        # 3.1 识别section行
        if is_section_row(row):
            current_section = extract_section_name(row['Name'])
            print(f"发现section: {current_section}")
            continue
        
        # 3.2 检查是否为有效股票行
        ticker, country_code = extract_ticker_and_country(row.get('Ticker'))
        
        # 跳过无效行
        if not ticker or (pd.isna(row.get('Name')) and pd.isna(row.get('Weight'))):
            continue
        
        # 3.3 构建结果记录
        results.append({
            'ticker': ticker,
            'source_etf': source_etf,
            'country_code': country_code,
            'region': '',  # 暂时留空
            'section': current_section or '',
            'weight': clean_numeric(row.get('Weight')),
            'shares': clean_numeric(row.get('Shares')),
            'price': clean_numeric(row.get('Price'))
        })
    
    # 4. 转换为DataFrame并保存
    result_df = pd.DataFrame(results)
    print(f"\n处理完成: {len(result_df)} 条有效记录")
    print(f"\n预览前5条:")
    print(result_df.head())
    
    # 5. 输出CSV
    result_df.to_csv(output_file, index=False)
    print(f"\n已保存到: {output_file}")
    
    return result_df

# ============= 执行 =============

if __name__ == '__main__':
    df_result = process_excel_to_csv(INPUT_FILE, OUTPUT_FILE, SOURCE_ETF)

读取文件: holdings.xlsx
原始数据: 10427 行
发现section: Information Technology
发现section: Financials
发现section: Industrials
发现section: Consumer Discretionary
发现section: Health Care
发现section: Communication Services
发现section: Consumer Staples
发现section: Materials
发现section: Energy
发现section: Utilities
发现section: Real Estate
发现section: N.A.

处理完成: 10414 条有效记录

预览前5条:
  ticker source_etf country_code region                 section    weight  \
0   NVDA        WLS           UW         Information Technology  4.330869   
1   MSFT        WLS           UW         Information Technology  3.723246   
2   AAPL        WLS           UW         Information Technology  3.562665   
3   AVGO        WLS           UW         Information Technology  1.526520   
4   2330        WLS           TT         Information Technology  0.837250   

      shares    price  
0  23444.971   192.57  
1   7429.881   522.40  
2  14619.625   254.04  
3   4612.342   345.02  
4  18536.260  1440.00  

已保存到: holdings_cleaned.csv


In [12]:
# 修正股数单位并重新计算市值
import pandas as pd

# 读取数据
df = pd.read_csv('holdings_cleaned.csv')

print("=== 修正股数单位 ===")
print("假设股数单位是千股，需要乘以1000")
print("例如：NVDA 23444.971 千股 = 23,444,971 股")

# 修正股数（乘以1000，从千股转为股）
df['shares_corrected'] = df['shares'] * 1000

# 重新计算市值
df['market_cap_corrected'] = df['shares_corrected'] * df['price']

print(f"\n=== 修正后的市值统计 ===")
print(df['market_cap_corrected'].describe())

print(f"\n=== 修正后的阈值分析 ===")
threshold_10b = 1e9  # 10亿
threshold_1b = 1e9    # 10亿

print(f"市值 > {threshold_10b/1e9:.0f}B 的股票数量: {len(df[df['market_cap_corrected'] > threshold_10b])}")
print(f"市值 > {threshold_1b/1e9:.0f}B 的股票数量: {len(df[df['market_cap_corrected'] > threshold_1b])}")

print(f"\n=== 修正后的最大市值股票 ===")
top_10 = df.nlargest(10, 'market_cap_corrected')[['ticker', 'shares_corrected', 'price', 'market_cap_corrected']]
print(top_10)

# 使用修正后的市值进行过滤
df_largecap = df[df['market_cap_corrected'] > threshold_10b].copy()
print(f"\n=== 最终结果 ===")
print(f"市值大于10B的股票数量: {len(df_largecap)}")

if len(df_largecap) > 0:
    print("\n前5只大市值股票:")
    print(df_largecap[['ticker', 'shares_corrected', 'price', 'market_cap_corrected']].head())
    
    # 保存结果
    df_largecap.to_csv('holdings_largecap_gt1bn.csv', index=False)
    print(f"\n已保存到 holdings_largecap_gt1bn.csv")
else:
    print("仍然没有找到市值大于10B的股票，可能需要进一步调整阈值或检查数据")


=== 修正股数单位 ===
假设股数单位是千股，需要乘以1000
例如：NVDA 23444.971 千股 = 23,444,971 股

=== 修正后的市值统计 ===
count    1.041400e+04
mean     6.981449e+08
std      8.243694e+09
min      4.939200e+04
25%      3.814589e+06
50%      1.286967e+07
75%      9.274314e+07
max      4.593896e+11
Name: market_cap_corrected, dtype: float64

=== 修正后的阈值分析 ===
市值 > 1B 的股票数量: 670
市值 > 1B 的股票数量: 670

=== 修正后的最大市值股票 ===
      ticker  shares_corrected     price  market_cap_corrected
9     005930         4866415.0   94400.0          4.593896e+11
1516    BBCA        52346754.0    7400.0          3.873660e+11
1571    BBRI        70121016.0    3730.0          2.615514e+11
28    000660          487029.0  428000.0          2.084484e+11
8098    TPIA        22219727.0    8100.0          1.799798e+11
9124    DSSA         1573762.0  106100.0          1.669761e+11
1668    BMRI        36538840.0    4250.0          1.552901e+11
9484    BREN        15549954.0    9850.0          1.531670e+11
6966    TLKM        47373012.0    3040.0          

In [ ]:
vgkTickers = [
  #  Netherlands
  'ASML', 'ASML.AS',
  #  Germany  
  'SAP', 'SAP.DE', 'SIE.DE', 'ALV.DE', 'DTE.DE', 'MBG.DE', 'BMW.DE', 'BAS.DE', 
  'VOW3.DE', 'ADS.DE', 'MUV2.DE', 'DB1.DE', 'CBK.DE', 'HEN3.DE', 'PAH3.DE',
  #  UK
  'AZN', 'AZN.L', 'SHEL.L', 'SHEL', 'HSBA.L', 'ULVR.L', 'DGE.L', 'BP.L', 
  'GSK.L', 'RIO.L', 'LSEG.L', 'NG.L', 'BARC.L', 'LLOY.L', 'VOD.L', 'PRU.L',
  'BT-A.L', 'REL.L', 'AV.L', 'RKT.L', 'IMB.L', 'BA.L', 'GLEN.L', 'AAL.L',
  'EXPN.L', 'CRDA.L', 'BDEV.L', 'RR.L', 'STAN.L', 'LGEN.L', 'INF.L', 'SSE.L',
  #  Switzerland
  'NESN.SW', 'NOVN.SW', 'ROG.SW', 'UHR.SW', 'UBSG.SW', 'ABBN.SW', 'ZURN.SW',
  'CSGN.SW', 'GIVN.SW', 'SREN.SW', 'SCMN.SW', 'LONN.SW', 'SLHN.SW', 'GEBN.SW',
  #  France
  'MC.PA', 'OR.PA', 'TTE.PA', 'SAN.PA', 'AIR.PA', 'BNP.PA', 'SAF.PA', 'SU.PA',
  'EL.PA', 'CS.PA', 'CAP.PA', 'DG.PA', 'BN.PA', 'VIV.PA', 'RI.PA', 'EN.PA',
  'SGO.PA', 'GLE.PA', 'ACA.PA', 'DSY.PA', 'KER.PA', 'RMS.PA', 'STM.PA', 'AI.PA',
  #  Denmark
  'NOVO-B.CO', 'ORSTED.CO', 'DSV.CO', 'MAERSK-B.CO', 'COLB.CO', 'DEMANT.CO',
  'GMAB.CO', 'VWS.CO', 'TRYG.CO', 'JYSK.CO',
  #  Spain
  'ITX.MC', 'IBE.MC', 'SAN.MC', 'BBVA.MC', 'TEF.MC', 'REP.MC', 'AMS.MC', 'ENG.MC',
  'FER.MC', 'IAG.MC', 'CABK.MC', 'SAB.MC', 'GRF.MC',
  #  Italy
  'ENI.MI', 'ISP.MI', 'ENEL.MI', 'UCG.MI', 'G.MI', 'TIT.MI', 'STM.MI', 'RACE.MI',
  'MB.MI', 'LDO.MI', 'BAMI.MI', 'TEN.MI', 'PRY.MI', 'CNHI.MI',
  #  Sweden
  'VOLV-B.ST', 'ERIC-B.ST', 'AZN.ST', 'ABB.ST', 'ATCO-A.ST', 'SEB-A.ST', 'HM-B.ST',
  'SAND.ST', 'SWED-A.ST', 'ASSA-B.ST', 'INVE-B.ST', 'HEXA-B.ST', 'ESSITY-B.ST',
  #  Norway
  'EQNR.OL', 'DNB.OL', 'MOWI.OL', 'TEL.OL', 'ORK.OL', 'YAR.OL', 'SALM.OL',
  #  Finland
  'NOKIA.HE', 'NESTE.HE', 'FORTUM.HE', 'STERV.HE', 'UPM.HE', 'SAMPO.HE',
  #  Belgium
  'ABI.BR', 'KBC.BR', 'UCB.BR', 'ACKB.BR', 'SOLB.BR',
  #  Ireland
  'CRH.I', 'SMDS.I', 'FLT.I', 'RYA.I', 'GLEN.I', 'CPL.I',
  #  Portugal
  'EDP.LS', 'GALP.LS', 'BCP.LS', 'JERONIMO.LS',
  #  Austria
  'OMV.VI', 'VOE.VI', 'ANDR.VI', 'POST.VI',
  #  Poland
  'PKO.WA', 'PZU.WA', 'PEO.WA', 'PKN.WA', 'KGH.WA',
  #  Greece
  'OPAP.AT', 'ALPHA.AT', 'ETE.AT', 'PPC.AT'
]



In [3]:
import pandas as pd

SOURCE_VGK = 'VGK'
REGION_VGK = 'developed'

# Export vgkTickers to CSV with required columns
output_path = '/Users/wenjiaqi/Downloads/bloomberg-competition/data/universes/vgk_tickers.csv'
df = pd.DataFrame({
    'ticker': vgkTickers,
    'source_etf': [SOURCE_VGK] * len(vgkTickers),
    'region': [REGION_VGK] * len(vgkTickers),
})
df.to_csv(output_path, index=False)
print(f"Saved {len(df)} rows to {output_path}")


Saved 178 rows to /Users/wenjiaqi/Downloads/bloomberg-competition/data/universes/vgk_tickers.csv


In [ ]:
# 改进的转换函数 - 提取country_code并优化ticker格式
def convert_complete_stock_data_improved(df):
    """
    将complete stock data转换为目标格式，提取country_code并优化ticker格式
    """
    # 创建新的DataFrame
    df_converted = pd.DataFrame()
    
    # 提取ticker和country_code
    def extract_ticker_and_country(ticker_text):
        """从 'NVDA US Equity' 提取 ('NVDA', 'US')"""
        if pd.isna(ticker_text):
            return None, None
        
        # 移除 ' Equity' 后缀
        clean_ticker = str(ticker_text).replace(' US Equity', '').replace(' Equity', '')
        
        # 分割获取ticker和country_code
        parts = clean_ticker.strip().split()
        if len(parts) >= 2:
            ticker = parts[0]
            country_code = parts[1]
        elif len(parts) == 1:
            ticker = parts[0]
            country_code = 'US'  # 默认为US
        else:
            return None, None
            
        return ticker, country_code
    
    # 应用提取函数
    ticker_country = df['Ticker'].apply(extract_ticker_and_country)
    df_converted['ticker'] = [x[0] for x in ticker_country]
    df_converted['country_code'] = [x[1] for x in ticker_country]
    
    # 添加其他列
    df_converted['source_etf'] = 'COMPLETE_DATA'
    df_converted['region'] = ''  # 暂时为空
    df_converted['section'] = 'Unknown'  # 可以根据需要设置
    df_converted['weight'] = np.nan
    df_converted['shares'] = np.nan
    df_converted['price'] = df['Price_D_1']
    
    # 添加原始数据中的其他有用列
    df_converted['market_cap'] = df['Market Cap _USD_']
    df_converted['p_b'] = df['P_B']
    df_converted['short_name'] = df['Short Name']
    df_converted['source_sheet'] = df['source_sheet']
    df_converted['row_index'] = df['row_index']
    
    # 移除无效行
    df_converted = df_converted.dropna(subset=['ticker'])
    
    return df_converted

# 执行改进的转换
df_converted_improved = convert_complete_stock_data_improved(df_complete)

print("改进转换后数据形状:", df_converted_improved.shape)
print("\n改进转换后数据列名:")
print(df_converted_improved.columns.tolist())
print("\n改进转换后前10行数据:")
df_converted_improved.head(10)


In [ ]:
# 检查country_code分布和ticker格式
print("=== Country Code 分布 ===")
print(df_converted_improved['country_code'].value_counts())

print("\n=== Ticker 格式示例 ===")
sample_tickers = df_converted_improved[['ticker', 'country_code', 'short_name']].head(15)
print(sample_tickers)

print("\n=== 为yfinance优化的ticker格式 ===")
# 为yfinance创建优化的ticker格式
def create_yfinance_ticker(ticker, country_code):
    """为yfinance创建合适的ticker格式"""
    if country_code == 'US':
        return ticker  # US股票直接使用ticker
    elif country_code == 'TT':  # 台湾
        return f"{ticker}.TW"
    elif country_code == 'UW':  # 可能是其他市场
        return ticker  # 暂时保持原样
    else:
        return f"{ticker}.{country_code}"  # 其他市场添加后缀

# 添加yfinance格式的ticker
df_converted_improved['yfinance_ticker'] = df_converted_improved.apply(
    lambda row: create_yfinance_ticker(row['ticker'], row['country_code']), axis=1
)

print("前10个yfinance格式的ticker:")
print(df_converted_improved[['ticker', 'country_code', 'yfinance_ticker']].head(10))


In [ ]:
# 保存改进后的转换数据
output_path_improved = '/Users/wenjiaqi/Downloads/bloomberg-competition/src/trading_system/data/complete_stock_data_converted_improved.csv'
df_converted_improved.to_csv(output_path_improved, index=False)

print(f"改进后的转换数据已保存到: {output_path_improved}")
print(f"文件大小: {len(df_converted_improved)} 行")

# 显示最终的数据结构
print("\n=== 最终数据结构 ===")
print(df_converted_improved.dtypes)

print("\n=== 数据统计 ===")
print(f"总行数: {len(df_converted_improved)}")
print(f"唯一ticker数: {df_converted_improved['ticker'].nunique()}")
print(f"唯一country_code数: {df_converted_improved['country_code'].nunique()}")

print("\n=== 最终数据示例 ===")
final_sample = df_converted_improved[['ticker', 'country_code', 'yfinance_ticker', 'price', 'market_cap', 'short_name']].head(10)
print(final_sample)
